In [ ]:
%%javascript
import { JupyterFrontEnd, JupyterFrontEndPlugin, ILabShell } from '@jupyterlab/application';

import { ICommandPalette, MainAreaWidget } from '@jupyterlab/apputils';

import { IDocumentManager } from '@jupyterlab/docmanager';

import { PageConfig } from '@jupyterlab/coreutils';

import { Message } from '@phosphor/messaging';

import { Widget } from '@phosphor/widgets';

interface IKubernetesCronJob {
  file_id: string;
  schedule: string;
}

class SchedulerWidget extends Widget {
  /**
   * Construct a new Kubernetes Scheduler widget.
   */
  constructor() {
    super();
    this._currentPath = '';

    this.addClass('scheduler-Widget');

    // Add a file ID element to the panel
    this._fileId = document.createElement('h1');
    this.node.appendChild(this._fileId);

    // Add an input element to the panel
    this._schedule = document.createElement('input');
    this.node.appendChild(this._schedule);

    // Add a button element to the panel
    this._button = document.createElement('button');
    this._button.addEventListener(
      'click',
      () => {
        if (this._button.innerText === 'Add') {
          this._addSchedule();
          return;
        }
        if (this._button.innerText === 'Delete') {
          this._deleteSchedule();
          return;
        }
    });
    this.node.appendChild(this._button);
  }

  private _currentPath: string;

  /**
   * The current widget path for the Scheduler widget.
   */
  set currentPath(value: string) {
    this._currentPath = value;
  }

  /**
   * The file ID text element associated with the Scheduler widget.
   */
  private _fileId: HTMLParagraphElement;

  /**
   * The schedule input element associated with the Scheduler widget.
   */
  private _schedule: HTMLInputElement;

  /**
   * The button element associated with the Scheduler widget.
   */
  private _button: HTMLButtonElement;

  private _setPanel(h1Text: string, inputValue: string) {
    this._fileId.innerText = h1Text;
    if (h1Text === 'ID: invalid') {
      this._schedule.value = '';
      this._schedule.disabled = true;
      this._button.innerText = 'Add';
      this._button.disabled = true;
      return;
    }
    if (inputValue === '') {
      this._schedule.value = inputValue;
      this._schedule.disabled = false;
      this._button.innerText = 'Add';
      this._button.disabled = false;
    } else {
      this._schedule.value = inputValue;
      this._schedule.disabled = true;
      this._button.innerText = 'Delete';
      this._button.disabled = false;
    }
  }

  private async _updatePanel(response: Response) {
    if (!response.ok) {
      return;
    }

    const k8sCronJob = await response.json() as IKubernetesCronJob;
    this._setPanel(`ID: ${k8sCronJob.file_id}`, k8sCronJob.schedule);
  }

  /**
   * Handle update requests for the Scheduler widget.
   */
  async onUpdateRequest(msg: Message): Promise<void> {
    if (!this._currentPath) {
      this._setPanel('ID: invalid', '');
      return;
    }

    const response = await fetch(`http://34.85.7.178:8888/show-info/${this._currentPath}`);
    if (!response.ok) {
      this._setPanel('ID: invalid', '');
      return;
    }

    const k8sCronJob = await response.json() as IKubernetesCronJob;
    this._setPanel(`ID: ${k8sCronJob.file_id}`, k8sCronJob.schedule);
  }

  private async _addSchedule() {
    if (!this._schedule.value || !this._currentPath) {
      return;
    }

    this._updatePanel(
      await fetch(`http://34.85.7.178:8888/add//${this._schedule.value}//${this._currentPath}`)
    );
  }

  private async _deleteSchedule() {
    if (!this._currentPath) {
      return;
    }

    this._updatePanel(
      await fetch(`http://34.85.7.178:8888/delete/${this._currentPath}`)
    );
  }
}

/**
 * Initialization data for the kube-scheduler extension.
 */
const extension: JupyterFrontEndPlugin<void> = {
  id: 'kube-scheduler',
  autoStart: true,
  requires: [ICommandPalette, IDocumentManager, ILabShell],
  activate: (
    app: JupyterFrontEnd, palette: ICommandPalette,
    docmanager: IDocumentManager, labShell: ILabShell
  ) => {
    console.log('JupyterLab extension kube-scheduler is activated!');
    console.log('JupyterLab server root is ' + PageConfig.getOption('serverRoot'));

    const content = new SchedulerWidget();
    const widget = new MainAreaWidget({content});
    widget.id = 'scheduler-jupyterlab';
    widget.title.label = 'k8s Schedule';

    const command: string = 'scheduler:open';
    app.commands.addCommand(command, {
      label: 'Kubernetes CronJob Scheduler',
      execute: () => {
        if (!widget.isAttached) {
          app.shell.add(widget, 'right');
        }
        const context = docmanager.contextForWidget(app.shell.currentWidget);
        if (context && context.path) {
            console.log('Current file path is ' + context.path);
            content.currentPath = context.path;
        } else {
            content.currentPath = '';
        }
        app.shell.activateById(widget.id);
      }
    });

    palette.addItem({ command, category: 'Kubernetes Operations' });
    app.shell.add(widget, 'right');

    // Change the schedule when the active widget changes.
    labShell.currentChanged.connect(() => {
      const currentWidget = app.shell.currentWidget;
      if (!currentWidget) {
        return;
      }
      const context = docmanager.contextForWidget(currentWidget);
      if (context && context.path) {
          console.log('Current file path is ' + context.path);
          content.currentPath = context.path;
      } else {
          content.currentPath = '';
      }
      widget.update();
    });
  }
};

export default extension;